# 09 CNN을 활용한 이미지 인식

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## 9.4 파이토치에서 CNN을 구현하는 방법

In [2]:
# nn.Cov2d(입력 채널수, 출력 채널수, 커널 사이즈)
conv1 = nn.Conv2d(3,32,3)
relu = nn.ReLU(inplace=True)
conv2 = nn.Conv2d(32,32,3)
maxpool = nn.MaxPool2d((2,2))

In [3]:
print(conv1)

print(conv1.weight.shape)
print(conv1.bias.shape)

print(conv2.weight.shape)
print(conv2.bias.shape)

Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([32, 32, 3, 3])
torch.Size([32])


In [4]:
# 입력용 더미 변수
inputs = torch.randn(100, 3, 32, 32)
print(inputs.shape)

torch.Size([100, 3, 32, 32])


In [5]:
x1 = conv1(inputs)
x2 = relu(x1)
x3 = conv2(x2)
x4 = relu(x3)
x5 = maxpool(x4)

In [6]:
print(inputs.shape)
print(x1.shape)
print(x2.shape)
print(x3.shape)
print(x4.shape)
print(x5.shape)

torch.Size([100, 3, 32, 32])
torch.Size([100, 32, 30, 30])
torch.Size([100, 32, 30, 30])
torch.Size([100, 32, 28, 28])
torch.Size([100, 32, 28, 28])
torch.Size([100, 32, 14, 14])


In [7]:
# nn.Sequential
features = nn.Sequential(
    conv1,
    relu,
    conv2,
    relu,
    maxpool
)

In [8]:
outputs = features(inputs)
print(outputs.shape)

torch.Size([100, 32, 14, 14])


In [9]:
# nn.Flatten
flatten = nn.Flatten()

outputs2 = flatten(outputs)

print(outputs.shape)
print(outputs2.shape)

torch.Size([100, 32, 14, 14])
torch.Size([100, 6272])


## 9.5 공통 함수 사용하기